# Intro to Policy Optimization

Reference: 
+ https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html
+ https://spinningup.openai.com/en/latest/algorithms/vpg.html 
+ Generalized Advantage Estimation (GAE) paper (ICLR2016): https://arxiv.org/pdf/1506.02438.pdf
+ Bias vs Variance in RL: https://blog.mlreview.com/making-sense-of-the-bias-variance-trade-off-in-deep-reinforcement-learning-79cf1e83d565
+ https://julien-vitay.net/deeprl/ActorCritic.html 

## I. Problem Formulation 
From the [Problem Formulation](docs/ProblemFormulation_Notation.ipynb), *the goal of RL* is to select an optimal policy $\pi^*$ which **maximizes expected return $J(\pi)$** when agent acts according to it:
    $$\pi^*= \argmax_\pi J(\pi) \quad \text{where} \quad J(\pi)= \mathbb{E}_{\tau \sim \pi} [R(\tau)] = \int_\tau P(\tau|\pi) R(\tau) \quad\quad (1)$$

where:
+ $R(\tau) = \sum_{t=0}^T \gamma^t r_t$ is the accumulated returns. 
+ $P(\tau|\pi)$ is the probability of a T-step trajectory $\tau=(s_0,a_0,s_1,a_1,...)$ and is derived as:
    $$P(\tau|\pi) = \rho_0(s_0) \prod_{t=0}^{T-1}P(s_{t+1}|s_t,a_t)\pi_\theta(a_t|s_t) \quad\quad (2)$$
  + the state transition $P(s_{t+1}|s_t,a_t)$ of the environment is **unknown**. 
  + $\pi_\theta(a_t|s_t)$ is the parameterized policy that we need to optimize through gradient descent(SGD): 
    $$\theta \leftarrow \theta + \nabla_\theta J(\pi_\theta)$$

## II. Solution 
To perform SGD update, we need to compute $\nabla_\theta J(\pi_\theta)$.
By definition and expand the expection into Integral, we have:
$$\nabla_\theta J(\pi_\theta) = \nabla_\theta \mathbb{E}_{\tau \sim \pi_\theta} [R(\tau)] = \nabla_\theta \int_\tau P(\tau|\pi_\theta) R(\tau)\quad\quad (3)$$
+ We can bring the Gradient $\nabla_\theta$ inside the Integral $\int_\tau$ because they operate with different variables $\theta$ and $\tau$. 
    $$\nabla_\theta J(\pi_\theta) =  \int_\tau \nabla_\theta P(\tau|\pi_\theta) R(\tau) \quad\quad (4) $$
+ Notice that, $\nabla_\theta P(\tau|\pi_\theta)$ is hard to compute directly, but $\nabla_\theta \log(P(\tau|\pi_\theta))$ is much easier. From (2), we have:
    $$\nabla_\theta \log(P(\tau|\pi_\theta)) = \nabla_\theta [\log \rho_0(s_0) + \sum_{t=0}^{T-1}\log P(s_{t+1}|s_t,a_t) + \sum_{t=0}^{T-1}\log\pi_\theta(a_t|s_t)]
                                             = \sum_{t=0}^{T-1}\nabla_\theta \log \pi_\theta(a_t|s_t) \quad\quad (5)$$
  because $\rho_0(s_0)$ and $P(s_{t+1}|s_t,a_t)$ are independent of $\theta$.
+ From the derivative of log function (a.k.a log-derivative trick), we also have
    $$\nabla_\theta \log(P(\tau|\pi_\theta)) = \frac{\nabla_\theta P(\tau|\pi_\theta)}{P(\tau|\pi_\theta)} \quad \Rightarrow \quad \nabla_\theta P(\tau|\pi_\theta)=P(\tau|\pi_\theta) \nabla_\theta \log(P(\tau|\pi_\theta)) \quad\quad (6)$$ 
+ Then, from (4),(5),(6) we have:
    $$ \nabla_\theta J(\pi_\theta) =  \int_\tau P(\tau|\pi_\theta) \sum_{t=0}^{T-1}\nabla_\theta \log \pi_\theta(a_t|s_t) R(\tau) = E_{\tau \sim \pi_{\theta}}[\sum_{t=0}^{T-1}\nabla_\theta \log \pi_\theta(a_t|s_t) R(\tau) ]\quad\quad (7)$$

In summary, (7) is the expectation, which can be implemented by sample mean:
+ We collect a set of $N$ trajectories $\mathcal{D} = \{\tau_i\}_{i=1,...,N}$ where each trajectory is obtained by letting the agent act in the environment using the policy $\pi_{\theta}$.
+ The policy gradient in (7) can be estimated with:
  $$ \nabla_\theta J(\pi_\theta) = \frac{1}{N} \sum_{\tau \in \mathcal{D}}\sum_{t=0}^{t \in \tau}\nabla_\theta \log \pi_\theta(a_t|s_t) R(\tau) \quad\quad (7) $$

##  III. Improving the Loss function:
**Problem:** From (7), the gradient of the Objective (loss) function is expressed as:
    $$ \nabla_\theta J(\pi_\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}}[\sum_{t=0}^{T-1}\nabla_\theta \log \pi_\theta(a_t|s_t) R(\tau) ] $$
It means that, taking a step with this gradient pushes up the log-probabilities of each action in proportion to $R(\tau)$, the sum of all rewards ever obtained.
Here, $R(\tau)$ is the weight for each action. However, the current formula uses the same weight for all action $a_t$, which is ineffective. 

**Modification**: Objective function (7) can be generalized as:
    $$ \nabla_\theta J(\pi_\theta) = \mathbb{E}_{\tau \sim \pi_{\theta}}[\sum_{t=0}^{T-1}\nabla_\theta \log \pi_\theta(a_t|s_t) \Phi_t ]\quad\quad (7)$$
where $\Phi_t$ is the modified weight, as long as it is not the function of $\theta$, e.g $\nabla_\theta \Phi_t=0$. Intuitively, we want $\Phi_t$ is large if the action $a_t$ leads to high returns, and vise versa. There can be many ways to define $\Phi_t$:
+ **Constant Return value**: by the default definition
  $$\Phi_t = R(\tau) = \sum_{t=0}^T \gamma^t r_t \quad \quad (8.a)$$
+ **Reward-to-go**, which is the return collected after the time $t$ only, since the weight of the action should only depend on the return achieved **after performing the action**, not before it: 
  $$\Phi_t = R_t = \sum_{t'\geq t} \gamma^{t'} r_{t'} \quad \quad (8.b)$$
+ **Baseline value**: We can inject the bias $b_\phi(s_t)$ function to the formular:
  $$\Phi_t = R_t - b(s_t)= \sum_{t'\geq t} \gamma^{t'} r_{t'} -b_\phi(s_t) \quad \quad (8.c)$$
+ **On-Policy Action-Value Function.**
  $$\Phi_t = Q^{\pi_\theta}(s_t,a_t)  \quad \quad (8.d) $$ 
+ **Advantage Function** describe how much better or worse it is than other actions on average.
  $$\Phi_t = A^{\pi_\theta}(s_t,a_t)=Q^{\pi_\theta}(s_t,a_t) - V^{\pi_\theta}(s_t) \quad \quad (8.e)$$ 
+ **Temporal Difference (TD) residual $\delta_t$**:
  $$\Phi_t = \rho_t = r_t + \gamma V^\pi(s_{t+1}) - V^\pi(s_t) \quad \quad (8.f)$$
+ **n-steps Temporal Difference (TD) residual**: by using the recursive formular $V(s_{t+1})=r_{t+1} + \gamma V(s_{t+2})$, we get 
  $$\Phi_t = \sum_{k=0}^{n-1} \gamma^k r_{t+k+1} + \gamma^n V^\pi(s_{t+k+1}) - V^\pi(s_t) = \sum_{k=0}^{n-1} \gamma^k \delta_{t+k} \quad \quad (8.f)$$

## IV. Generalized Advantage Estimation (GAE)
**Q1: There are many options for $\Phi_t$, what should we choose?** General speaking,
+ The Constant Reward (8.a), Reward-To-Go (8.b) and Baseline Value (8.c) are directltly dependent on the collected rewards in a single episode, so they are strongest feedback to improve the policy. However, because the environment and the policy are both stochastic, using the raw rewards leads to high variance (two trajectories starting from the same state can arrive with very different results). Thus, to make it work, we must take the average of many trajectories, which leads to very slow convergence. 
+ In constrast, using Action-Value Function (8.d), Advantage Function (8.e), and TD residual (8.f) requires us to approximate the $V^\pi(s)$ or $Q^\pi(s,a)$ functions, because their true value are unknown by definition. Commonly, we approximate it by a critic network $\{\phi\}$: 
    $$ \phi \leftarrow \phi + \nabla_\phi (V^\phi(s,a)-R(\tau))^2 $$
  By doing so, we can update the policy every single step, which is very efficient. However, because the function $V^\phi$ is estimated by a small set of data that we collected, it has high bias, which leads to inaccurate estimation of $V^\pi$ and consequently sub-optimal policy.  
+ The n-steps TD residual (8.f) offers the good trade-off between the two tendence. The larger $n$, the less $V^phi$ can contribute due to $\gamma^n$ and the more direct reward $r_t$ are used. Consequently, the less bias but the more variance it will induce. This is used in the Advantage Actor-Critic (A2C) algorithm. 

**Q2: Then, the next question is how to choose $n$?** 

The **Generalized Advantage Estimation (GAE)** proposes to use all the steps $n=1:\infty$, but with exponential weights $\lambda$. Let $A_t^n$ be the estimated advantage using a single $n$ step. 
  $$A_t^n = \sum_{k=0}^{n-1} \gamma^k \delta_{t+k} \quad \quad (9)$$
then:
  \begin{align}
  \Phi_t^{GAE} &= (1-\lambda)[A_t^1 + \lambda A_t^2 + \lambda^2 A_t^3 + \cdots + \lambda^\infty A_t^\infty] \\
         &= (1-\lambda)[\delta_t + \lambda (\delta_t + \gamma \delta_{t+1} ) + \lambda^2(\delta_t + \gamma \delta_{t+1} + \gamma^2 \delta_{t+2}) + \cdots] \\
         &= (1-\lambda)[\delta_t(1 + \lambda + \lambda^2 + \cdots) + \gamma \delta_{t+1}(\lambda + \lambda^2 + \lambda^3 + \cdots) + \cdots] \\
         &= (1-\lambda)[\delta_t \frac{1}{1-\lambda} + \lambda \gamma \delta_{t+1} \frac{1}{1-\lambda} + \cdots] \\
         &= \delta_t + \lambda \gamma \delta_{t+1} + \lambda^2 \gamma^2 \delta_{t+2} + \cdots = \sum_{n=0}^\infty (\lambda \gamma)^n \delta_{t+n}\quad \quad (10)
  \end{align}
For $N$ large enough, $10$ can be approximate by  $\Phi_t^{GAE}=\sum_{n=0}^N (\lambda \gamma)^n \delta_{t+n}$. 

## V. Vanila Policy Optimization (VPG)
VPG uses the Advantage Function as the weight, where the pseudo code is as follow:

<img src="https://spinningup.openai.com/en/latest/_images/math/262538f3077a7be8ce89066abbab523575132996.svg"
     alt="Vanilla Policy Optimization"
     width="700"
     style="float: left; margin-right: 20px;" />